<a href="https://colab.research.google.com/github/Prakhar-porwal/local-voice-tts-clean/blob/main/notebooks/fast_tts_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Local Voice TTS - Turbo Mode (Google Colab)

This notebook runs the **Local Voice TTS Backend** on a **Free Google Colab T4 GPU**.
This is roughly **10x-20x faster** than the Hugging Face CPU Free Tier.

### Instructions:
1.  **Runtime** -> **Change runtime type** -> **T4 GPU** (should be default).
2.  **Runtime** -> **Disconnect and Delete Runtime** (If retrying).
3.  Run **Cell 1** (Setup).
4.  **Restart Session** (When prompted by Cell 1).
5.  Run **Cell 2** (Start Server).
6.  Copy the **TURBO API URL** into your Vercel App Settings.

In [ ]:
# @title 1. Setup & Install (Correct Order)
import os

# 1. Clean previous runs
if os.path.exists("local-voice-tts-clean"):
    !rm -rf local-voice-tts-clean

# 2. Clone Repository
!git clone https://github.com/Prakhar-porwal/local-voice-tts-clean
%cd local-voice-tts-clean

# 3. System Dependencies
!apt-get -y install espeak-ng libsndfile1

# 4. Install Build Tools first
!pip install hatchling cython numpy==1.26.4

# 5. Install TTS (THIS UPGRADES TRANSFORMERS, SO WE DO IT FIRST)
print("Installing TTS...")
!pip install --no-build-isolation "git+https://github.com/idiap/coqui-ai-TTS.git#egg=coqui-tts"

# 6. OVERRIDE with Correct Dependencies
print("🔧 Forcing Correct Package Versions...")
!pip install --force-reinstall transformers==4.40.0 accelerate==0.29.0
!pip install --force-reinstall "numpy<2.0" "scipy" setuptools wheel

# 7. Install Server Deps
!pip install fastapi "uvicorn[standard]" librosa soundfile pydub python-multipart pyngrok nest_asyncio

print("\n✅ Setup Complete. Please RESTART RUNTIME & RUN CELL 2.")


In [ ]:
# @title 2. Start Turbo Backend
import os
import subprocess
import sys
import time
import transformers

print(f"🧐 Transformers Version: {transformers.__version__} (Should be 4.40.0)")

# --- 1. APPLY HOTFIX ---
target_file = "/usr/local/lib/python3.12/dist-packages/TTS/tts/layers/tortoise/autoregressive.py"
if os.path.exists(target_file):
    with open(target_file, "r") as f:
        content = f.read()
    if "isin_mps_friendly" in content:
        print("🔧 Patching TTS Library...")
        new_content = content.replace("from transformers.pytorch_utils import isin_mps_friendly as isin", "from torch import isin")
        with open(target_file, "w") as f:
            f.write(new_content)

# --- 2. PATCH main.py (GPU) ---
main_file = "/content/local-voice-tts-clean/main.py"
if os.path.exists(main_file):
    print("🔧 Patching main.py for CUDA/GPU Support...")
    with open(main_file, "r") as f:
        code = f.read()
    code = code.replace('if torch.backends.mps.is_available():\n    device = "mps"\nelse:\n    device = "cpu"', 'device = "cuda" if torch.cuda.is_available() else "cpu"')
    code = code.replace('xtts_tts = TTS(XTTS_MODEL).to("cpu")', 'xtts_tts = TTS(XTTS_MODEL).to(device)')
    with open(main_file, "w") as f:
        f.write(code)
    print("✅ GPU Patch Applied!")

# --- 3. START SERVER ---
print("🚀 Starting Server...")
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Auto-Accept License
os.environ["COQUI_TOS_AGREED"] = "1"
# Force GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Reset Ngrok
ngrok.kill()
# 🔑 SET AUTH TOKEN
ngrok.set_auth_token("36jQzzt9HifzkJJKUwtGTOAt3nA_3hZTGahebR2E5xk4WDX7o")
nest_asyncio.apply()

try:
    print("⏳ Connecting to Ngrok...")
    time.sleep(2)
    public_url = ngrok.connect(7860).public_url
    print("\n================================================================")
    print(f"🚀 TURBO API URL: {public_url}")
    print("================================================================\n")
except Exception as e:
    print(f"❌ Ngrok Error: {e}")
    print("⚠️ IF THIS SAYS 'ERR_NGROK_108': Your account has too many active sessions.")
    print("   -> Go to dashboard.ngrok.com and kill old agents, OR use a new token.")

if os.path.exists("/content/local-voice-tts-clean"):
    os.chdir("/content/local-voice-tts-clean")

config = uvicorn.Config("main:app", host="127.0.0.1", port=7860)
server = uvicorn.Server(config)
import asyncio
await server.serve()